In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.datasets import make_moons
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout, Concatenate, Lambda, GaussianNoise, Activation, GaussianDropout
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.losses import BinaryCrossentropy, Huber
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental.preprocessing import Normalization
#import tensorflow as tf
#import numpy as np
import pandas as pd
from tqdm import tqdm
from random import choices

from tensorflow.keras.constraints import max_norm, min_max_norm
from keras.callbacks import ReduceLROnPlateau
import os

#from distutils.dir_util import copy_tree
import shutil


In [ ]:
train = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
SEED = 1111

tf.random.set_seed(SEED)
np.random.seed(SEED)

In [ ]:
train_part = train
#### CHANGE This back to impute strategy...
#train_part = train_part.fillna(0) # y4es you need this
train_part_mean = train_part.mean()
train_part_std = train_part.std()
train_part_var = train_part.var()
train_part.fillna(train_part_mean,inplace=True)
train_part['action'] = ((train_part['resp'].values) > 0).astype(int)

features = [c for c in train_part.columns if "feature" in c]
features = features + ['resp_1'] + ['resp_2'] + ['resp_3'] + ['resp_4'] + ['resp'] + ['weight']  + ['action'] + ['date']

train_part = train_part[features]


In [ ]:
train_part.shape

In [ ]:
train_part_mean


In [ ]:
train_part_std

In [ ]:
train_part_var

In [ ]:
train_part = tf.convert_to_tensor(
    train_part, dtype="float32", dtype_hint=None, name=None
)

In [ ]:
normalized_data_test=train_part # notactually normalized, but lets see if this makes life easier.

In [ ]:
#norm = layers.experimental.preprocessing.Normalization(mean=train_part_mean, var=train_part_var)
#layers.experimental.preprocessing.Normalization(axis=-1, dtype=None, mean=None, variance=None)

#tf.keras.layers.experimental.preprocessing.Normalization(
#    mean=3., variance=2.
#)

#adapt_data = np.array([[1.], [2.], [3.], [4.], [5.]], dtype=np.float32)
#input_data = np.array([[1.], [2.], [3.]], np.float32)
#layer = Normalization(mean=3., variance=2.)
#layer.adapt(adapt_data)
#layer(input_data)

In [ ]:




#norm.adapt(train_part)
#normalized_data_test = norm(train_part)
#normalized_data_test

In [ ]:
### CHANGE THIS BELOW for it to work based on dims of input....
#dim_shape=4
dim_shape = 138 # this works 
#dim_shape = 136 # this works 
#dim_shape = 2 # this works 

# Creating a custom layer with keras API.
#output_dim = 256
output_dim = 138 #not sure which one to do
reg = 0.01


def Coupling(input_shape):
    input = keras.layers.Input(shape=input_shape)

    t_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    t_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_1)
    t_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_2)
    t_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_3)
    t_layer_5 = keras.layers.Dense(
        input_shape, activation="linear", kernel_regularizer=regularizers.l2(reg)
    )(t_layer_4)

    s_layer_1 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(input)
    s_layer_2 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_1)
    s_layer_3 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_2)
    s_layer_4 = keras.layers.Dense(
        output_dim, activation="relu", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_3)
    s_layer_5 = keras.layers.Dense(
        input_shape, activation="tanh", kernel_regularizer=regularizers.l2(reg)
    )(s_layer_4)

    return keras.Model(inputs=input, outputs=[s_layer_5, t_layer_5])

class RealNVP(keras.Model):
    def __init__(self, num_coupling_layers):
        super(RealNVP, self).__init__()

        self.num_coupling_layers = num_coupling_layers

        # Distribution of the latent space.
#        self.distribution = tfp.distributions.MultivariateNormalDiag(
#            loc=[0.0, 0.0], scale_diag=[1.0, 1.0]
#        )
        self.distribution = tfp.distributions.MultivariateNormalDiag(
            loc=np.zeros(dim_shape).tolist(), scale_diag=np.ones(dim_shape).tolist()
#            loc=np.zeros(dim_shape).tolist(), scale_diag=np.ones(3).tolist()
#            loc=np.zeros(3).tolist(), scale_diag=np.ones(3).tolist()
        )

#        self.masks = np.array(
#            [[0, 1], [1, 0]] * (num_coupling_layers // 2), dtype="float32"
#        )
        # need mto make this dynamic e.g. identiy(3) will be euiqvalen to size ofrectangle
    # also need to check this against original moons data with size of 2 to see if it works..
        self.masks = np.array(
            np.identity(dim_shape).tolist() * (num_coupling_layers // 2), dtype="float32"
#            np.identity(3).tolist() * (num_coupling_layers // 2), dtype="float32"
        )
        self.loss_tracker = keras.metrics.Mean(name="loss")
        self.layers_list = [Coupling(dim_shape) for i in range(num_coupling_layers)]
#        self.layers_list = [Coupling(3) for i in range(num_coupling_layers)]

    @property
    def metrics(self):
        """List of the model's metrics.
        We make sure the loss tracker is listed as part of `model.metrics`
        so that `fit()` and `evaluate()` are able to `reset()` the loss tracker
        at the start of each epoch and at the start of an `evaluate()` call.
        """
        return [self.loss_tracker]

    def call(self, x, training=True):
        log_det_inv = 0
        direction = 1
        if training:
            direction = -1
        for i in range(self.num_coupling_layers)[::direction]:
#            import pdb;pdb.set_trace()
            x_masked = x * self.masks[i]
#            x_masked = x * self.masks[i]
            reversed_mask = 1 - self.masks[i]
#            import pdb;pdb.set_trace()
            s, t = self.layers_list[i](x_masked)
            s *= reversed_mask
            t *= reversed_mask
            gate = (direction - 1) / 2
            x = (
                reversed_mask
                * (x * tf.exp(direction * s) + direction * t * tf.exp(gate * s))
                + x_masked
            )
            log_det_inv += gate * tf.reduce_sum(s, [1])

        return x, log_det_inv

    # Log likelihood of the normal distribution plus the log determinant of the jacobian.

    def log_loss(self, x):
        y, logdet = self(x)
#        import pdb;pdb.set_trace()
        log_likelihood = self.distribution.log_prob(y) + logdet
        return -tf.reduce_mean(log_likelihood)

    def train_step(self, data):
        with tf.GradientTape() as tape:

            loss = self.log_loss(data)

        g = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(g, self.trainable_variables))
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}

    def test_step(self, data):
        loss = self.log_loss(data)
        self.loss_tracker.update_state(loss)

        return {"loss": self.loss_tracker.result()}

    


In [ ]:
#!cp -r ../input/keras-normalizing-flow-backup/training_PRODUCTION/ ./training_PRODUCTION/

In [ ]:
#!mkdir training_PRODUCTION/
#!cp /kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index
#!cp /kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001 training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001
#!cp /kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index
#!dir training_PRODUCTION

In [ ]:
try:
    !mkdir training_PRODUCTION/
except:
    print('couldnt make directory')

try:
# issues with recursion previously, so going to just copy what is necessary...
#    !cp -r /kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/ training_PRODUCTION/
    !cp /kaggle/input/nvp-impute/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index
    !cp /kaggle/input/nvp-impute/training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001 training_PRODUCTION/cp_PRODUCTION.ckpt.data-00000-of-00001
    !cp /kaggle/input/nvp-impute/training_PRODUCTION/cp_PRODUCTION.ckpt.index training_PRODUCTION/cp_PRODUCTION.ckpt.index

except:
    print('error in copying over.')

In [ ]:

if(len(normalized_data_test)==2390491):
    checkpoint_path = "training_PRODUCTION/cp_PRODUCTION.ckpt"
    checkpoint_path_read = "/kaggle/input/nvp-impute/training_PRODUCTION/cp_PRODUCTION.ckpt"
    checkpoint_path_read_fulldir = "/kaggle/input/nvp-impute/training_PRODUCTION/"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)
else:
    checkpoint_path = "training_QA/cp_QA.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    # Create a callback that saves the model's weights
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                     save_weights_only=True,
                                                     verbose=1)



In [ ]:
# just keep running this, and every say so many times create a backup, like nightly backup
model_chk = RealNVP(num_coupling_layers=6)
model_chk.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001))
# 3 minutes to evaluate
model_chk.evaluate(normalized_data_test)
try:
    model_chk.load_weights(checkpoint_path_read)
    # 3 minutes to evaluate
    model_chk.evaluate(normalized_data_test)
    model_chk.fit(
    normalized_data_test, batch_size=256, epochs=30, verbose=2, validation_split=0.2, callbacks = [cp_callback]
    )
except:
    print('failed going to refit the model, careful as you can lose all progress')
#    print('going to just fail so we dont mess something up for now...things should have been copied over already')
#except:
    model_chk.fit(
    normalized_data_test, batch_size=256, epochs=1, verbose=2, validation_split=0.2, callbacks = [cp_callback]
    )

In [ ]:
### save out the normalized data as well so you don't have to keep lodaing it in, maybe it will load faster...
# Epoch 00015: saving model to training_PRODUCTION/cp_PRODUCTION.ckpt
#7471/7471 - 297s - loss: 128.7271 - val_loss: 120.2163
        

In [ ]:
#!ls -l "/kaggle/input/keras-normalizing-flow-backup/training_PRODUCTION/"

In [ ]:
#!ls -l "training_PRODUCTION/"#
#!cat "training_PRODUCTION/checkpoint"

In [ ]:
#normalized_data_part = normalized_data_test[0:5]


#print(normalized_data_part)
# From data to latent space.
#z, _ = model(normalized_data_part)
#print(z)

#From latent space to data.
#samples = model.distribution.sample(3000)
#x, _ = model.predict(samples)


In [ ]:
# need to write code to autoarchive the model weights so have a backup ever say 50 or so runs...
# e.g. backup_training_production_1/*
#backup_training_production_2/
#!mkdir training_prod_backup_20200223

#!cp -r training_PRODUCTION 
#!dir training_PRODUCTION
#!dir training_prod_backup_20200223

In [ ]:
#!dir training_prod_backup_20200223/training_PRODUCTION